# CNN Model Pipeline

## Imports Libraries

In [16]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from src.paths import *

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization
)

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## Setup Data

In [17]:
train_dir = data_dir / "Train"
test_dir = data_dir / "Test"
train_csv_path = data_dir / "Train.csv"

## Prepare Data Generator

In [18]:
data_generator = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2,
)

train_generator = data_generator.flow_from_directory(
    directory=train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    seed=42
)

validation_generator = data_generator.flow_from_directory(
    directory=train_dir,
    target_size=(32, 32),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    seed=42
)

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


## Build CNN Model

In [19]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(43, activation='softmax')
])

## Model Compilation & Callbacks Set Up

In [21]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint(filepath=(model_dir / "best_model.keras"), save_best_only=True)
]

## Train Model

In [22]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.4377 - loss: 4.6910 - val_accuracy: 0.8319 - val_loss: 0.7633
Epoch 2/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.8920 - loss: 0.5444 - val_accuracy: 0.8688 - val_loss: 0.8823
Epoch 3/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.9356 - loss: 0.3079 - val_accuracy: 0.8689 - val_loss: 0.8327
Epoch 4/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9465 - loss: 0.2705 - val_accuracy: 0.8982 - val_loss: 0.7205
Epoch 5/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9557 - loss: 0.2364 - val_accuracy: 0.9073 - val_loss: 0.9374
Epoch 6/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9580 - loss: 0.2536 - val_accuracy: 0.9089 - val_loss: 0.7183
Epoch 7/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9706 - loss: 0.1788 - val_accuracy: 0.9183 - val_loss: 0.9979
Epoch 8/15
981/981 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.9662 - loss: 0.2158 - 

## Show Model Summary

In [23]:
# Show Model Summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 960,707 (3.66 MB)

 Trainable params: 320,171 (1.22 MB)

 Non-trainable params: 192 (768.00 B)

 Optimizer params: 640,344 (2.44 MB)